In [15]:
import numpy as np
from numpy import log, exp
from scipy.optimize import minimize


In [16]:
def gme_support_vector(nobs):
    return np.array([-1.0/np.sqrt(nobs), 0.0, 1.0/np.sqrt(nobs)])


In [17]:

def me_discrete_gme_binary(params, Y, X, v):
    """
    A simplified GME objective for a binary outcome Y in {0,1}.
    Y is length-N, X is NxK, params is length-K.
    v is the 3-point error support.
    Returns the *negative* of the objective, so SciPy can minimize it.
    """

    # 1) Compute linear index
    P1 = X @ params  # shape (N,)

    # 2) We'll define:
    #    a) sum( P1 * Y ), 
    #    b) sum( ln( sum_{k in 2-cat} exp(-P1_k) ) ), but we only have one cat for "Y=1"
    #       so let's adapt.
    #
    # Actually, if you want to replicate exactly the multi-col approach, you'd do a
    # custom sum over categories. But let's do a simpler approach:
    
    # The usual GME logic might be:
    # L = - [ sum(Y_i * P1_i) + sum( ln( sum_{k} exp(-P1_{i,k}) ) ) + sum( ln( ... PSI... )) ]
    # But if we have only 2 categories => P(1) = p, P(0) = 1-p
    # GME code might do something special with error terms. We'll do a simpler version:

    # Let's define p = exp(-P1) / [1 + exp(-P1)] to mimic your final "predict" formula
    # Then define the negative log-likelihood-like term:
    # part1 = sum(Y_i * log(p_i) + (1-Y_i)*log(1-p_i))   # for a standard logit
    # Then we incorporate GME "penalties" or "entropy" terms as the code does.

    # For demonstration, let's show a direct replication of your final lines from gme_discrete:
    # P = exp(-X*beta')/(1 + exp(-X*beta'))
    # Then an "entropy" measure S = ...
    # But the original code uses a 3-point error structure. 
    # We'll keep it minimal here:

    p = np.clip(np.exp(-P1)/(1.0 + np.exp(-P1)), 1e-15, 1-1e-15)
    # Negative log-likelihood part:
    nll = -np.sum(Y * np.log(p) + (1 - Y)*np.log(1 - p))

    # If you want to incorporate the "v" error support, you'd do so similarly to your original:
    # For example, you might define something akin to: sum(log( sum exp( - P1_i v ) ) ) etc.
    # We'll omit that for brevity here. If you need the exact 3-point logic, carefully re-code.

    return nll


In [18]:

def gme_logit_binary(y, X):
    """
    A single-column parameter approach for a binary GME logit,
    with fewer chances of overflow.
    """
    y = np.array(y).astype(float)
    X = np.array(X).astype(float)
    N, K = X.shape
    v = gme_support_vector(N)
    init_params = np.zeros(K)

    def obj_fun(params):
        return me_discrete_gme_binary(params, y, X, v)

    res = minimize(obj_fun, init_params, method='BFGS')
    return res


In [19]:
def add_constant(X):
    """
    Append a column of ones to X for the intercept.
    """
    X = np.asarray(X).astype(float)
    ones = np.ones((X.shape[0], 1))
    return np.hstack((X, ones))


In [11]:
def gme_support_vector(nobs):
    """
    Symmetric 3-point error support: [-1/sqrt(N), 0, 1/sqrt(N)].
    """
    return np.array([-1.0/np.sqrt(nobs), 0.0, 1.0/np.sqrt(nobs)])


In [21]:
if __name__ == "__main__":
    # Generate a small synthetic dataset
    np.random.seed(123)
    N = 200
    Xraw = np.random.randn(N, 2)  # two regressors
    # True betas
    b_true = np.array([1.0, -1.5, 0.5])  # 2 slopes + intercept
    # We'll do a logistic transformation for 'true' probabilities:
    X_ = add_constant(Xraw)  # shape (N, 3)
    xb = X_ @ b_true
    p_true = 1.0/(1.0 + np.exp(-xb))
    y_obs = (np.random.rand(N) < p_true).astype(float)

    # Run gmentropylogit
    out = gme_logit_binary(y_obs, Xraw)
    print(out)
#    print("Converged:", out['success'])
#    print("Parameters (K x 2):\n", out['params'])
#    print("Entropy:", out['entropy'], "Normalized entropy (Sp):", out['Sp'])
#    print("lnf:", out['lnf'])
#    print("Partial effects:", out['mfx'])


  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 90.64208733063595
        x: [-1.119e+00  1.764e+00]
      nit: 10
      jac: [ 0.000e+00  0.000e+00]
 hess_inv: [[ 4.570e-02 -2.222e-02]
            [-2.222e-02  7.580e-02]]
     nfev: 36
     njev: 12
